In [ ]:
pip install ultralytics opencv-python-headless

In [ ]:
import os
from ultralytics import YOLO
import cv2  # To load the image and get dimensions

# Load YOLO model
model = YOLO("yolov8n.pt")

def is_already_labeled(x, y, w, h, existing_labels):
    for label in existing_labels:
        lx, ly, lw, lh = map(float, label[1:])
        if abs(x - lx) < lw / 2 and abs(y - ly) < lh / 2:
            return True
    return False

def normalize_coordinates(x, y, w, h, img_width, img_height):
    x_center = x / img_width
    y_center = y / img_height
    width = w / img_width
    height = h / img_height
    return x_center, y_center, width, height

def process_image(image_path, label_path):
    # Load existing labels
    with open(label_path, 'r') as f:
        existing_labels = [line.strip().split() for line in f.readlines()]

    # Load image and get its dimensions
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # (height, width)

    results = model.predict(source=image_path, save=False, conf=0.23)[0]
    updated_labels = existing_labels.copy()

    # Iterate through detected bounding boxes
    for box, cls in zip(results.boxes.xywh, results.boxes.cls):
        x, y, w, h = box[:4].tolist()  # Convert tensor to list
        cls = int(cls.item())  # Convert tensor to int for class ID
        
        if cls == 0:  # Assuming YOLO class 0 represents "person"
            # Normalize the bounding box coordinates
            x_center, y_center, width, height = normalize_coordinates(x, y, w, h, img_width, img_height)

            # Check if the object is already labeled
            if not is_already_labeled(x_center, y_center, width, height, existing_labels):
                updated_labels.append([3, x_center, y_center, width, height])  # Assign '3' to new students

    # Write updated labels back to the label file
    with open(label_path, 'w') as f:
        for label in updated_labels:
            f.write(' '.join(map(str, label)) + '\n')

# Define paths for images and labels
image_folder = os.path.join(".", "dataset", "images", "train")
label_folder = os.path.join(".", "dataset", "labels", "train")

# Iterate through all images and their corresponding labels
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):
        image_path = os.path.join(image_folder, filename)
        label_path = os.path.join(label_folder, filename.replace(".jpg", ".txt"))

        if os.path.exists(label_path):
            process_image(image_path, label_path)
            print(f"Processed and updated: {filename}")
        else:
            print(f"Label file missing for: {filename}")



image 1/1 C:\Users\Acer\Project_Classroom\Real_classroom2\dataset\images\train\0001003.jpg: 448x640 12 persons, 1 chair, 1 tv, 132.7ms
Speed: 4.0ms preprocess, 132.7ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)
Processed and updated: 0001003.jpg

image 1/1 C:\Users\Acer\Project_Classroom\Real_classroom2\dataset\images\train\0001004.jpg: 384x640 14 persons, 2 cups, 2 dining tables, 120.9ms
Speed: 2.0ms preprocess, 120.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed and updated: 0001004.jpg

image 1/1 C:\Users\Acer\Project_Classroom\Real_classroom2\dataset\images\train\0001005.jpg: 384x640 16 persons, 2 cups, 1 chair, 2 dining tables, 1 laptop, 103.1ms
Speed: 2.0ms preprocess, 103.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed and updated: 0001005.jpg

image 1/1 C:\Users\Acer\Project_Classroom\Real_classroom2\dataset\images\train\0001006.jpg: 384x640 17 persons, 2 cups, 2 dining tables, 2 laptops, 147.5